# Compare Unets with 2 inputs

Vanilla u net  
Unet with 2 inputs  
Unet with 2 inputs on different depth  
Double U net  
Triple U net

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/M_DV_V2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input 
from keras.utils.vis_utils import plot_model

from models.vanilla_unet import vanilla_unet
from models.unet_2input import unet_2input
from models.unet_2input_deep import unet_2input_deep
from models.double_unet import double_unet
from models.triple_unet import triple_unet

### All nets

In [3]:
seed = 123

vUnet = vanilla_unet(seed)
unet2i = unet_2input(seed)
unet2i_d = unet_2input_deep(seed)
dUnet = double_unet(seed)
tUnet = triple_unet(seed)

### Import data set and show

In [ ]:
X_train = np.load("data/u_net_roofs/X_data.npy") 
X_70_train = np.load("data/u_net_roofs/X_70_data.npy")
y_train = np.load("data/u_net_roofs/y_data.npy")

X_val = np.load("data/u_net_roofs/X_data_val.npy")
X_70_val = np.load("data/u_net_roofs/X_70_data_val.npy")
y_val = np.load("data/u_net_roofs/y_data_val.npy")

X_test = np.load("data/u_net_roofs/X_data_test.npy")
X_70_test = np.load("data/u_net_roofs/X_70_data_test.npy")
y_test = np.load("data/u_net_roofs/y_data_test.npy")

### Set up nets

In [27]:
input_img1 = Input(shape=(128,128,177))
input_img2 = Input(shape=(56,56,200))

model = dUnet.get_unet(input_img1,input_img2, n_classes=2, last_activation='softmax')

model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

## Run tests

In [5]:
h = model.fit([X_train, X_70_train],
               y_train, 
               validation_data=([X_val, X_70_val],y_val), 
               batch_size=16,
               epochs=300)

NameError: name 'X_train' is not defined

## Visualize model

In [28]:
plot_model(model, to_file="doubleUnet.png", dpi=300, rankdir='TB')